In [1]:
!pip install langchain openai


  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.2 MB/s eta 0:00:00a 0:00:01
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uni

In [18]:
from langchain.tools import Tool
import requests
from bs4 import BeautifulSoup
def calculator_tool(x):
    import re
    x = x.lower().strip()
    numbers = list(map(float, re.findall(r"\d+(?:\.\d+)?", x)))

    if "add" in x or "+" in x or (len(numbers) == 2 and "then" not in x):
        return str(sum(numbers))
    elif "subtract" in x or "-" in x:
        return str(numbers[0] - numbers[1])
    elif "multiply" in x or "*" in x:
        return str(numbers[0] * numbers[1])
    elif "divide" in x or "/" in x:
        return str(numbers[0] / numbers[1])
    return "Unsupported operation"





In [37]:
import requests
from bs4 import BeautifulSoup

def web_scrape_limited(input_str):
    try:
        import re

        parts = input_str.strip().split()
        url = next((p for p in parts if p.startswith("http")), None)
        if url:
            url = url.strip("'\"")  # strip any trailing quotes

        lines = int(re.findall(r'\d+', input_str)[-1])

        if not url:
            return "No valid URL found in input."

        response = requests.get(url)
        if response.status_code != 200:
            return f"Failed to fetch page, status code: {response.status_code}"

        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = '\n'.join(p.get_text() for p in paragraphs)
        limited_text = "\n".join(text.splitlines()[:lines])
        return limited_text if limited_text else "No lines found."

    except Exception as e:
        return f"Error during scraping: {str(e)}"



calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Performs basic arithmetic operations like add, subtract, multiply, divide. Input should be natural language like 'Add 100 and 5'."
)
web_scraper_limited = Tool(
    name="Web Scraper",
    func=web_scrape_limited,
    description="Scrapes the specified number of lines from a web page. Input should be like 'Scrape 105 lines from https://example.com'"
)


In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)


In [39]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[calculator, web_scraper_limited],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [40]:
response = agent.run(
    "Add 100 and 5, then scrape that many lines from https://edwarddonner.com"
)
print(response)



> Entering new AgentExecutor chain...
First, I need to add 100 and 5 using the calculator tool.
Action: Calculator
Action Input: 'Add 100 and 5'
Observation: 105.0
Thought:Now that I have the sum, I need to scrape 105 lines from the specified website.
Action: Web Scraper
Action Input: 'Scrape 105 lines from https://edwarddonner.com'
Observation: I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (very amateur) and losing myself in Hacker News, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of Nebula.io. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt, acquired in 2021.
We wo